## 4-1. 프로젝트: 인물 모드 문제점 찾기

만들면서 뭔가 이상한 점이 느껴지지 않으셨나요? 프로젝트를 통해 찾아봅시다.

우선 주요 라이브러리 버전을 확인해 볼까요?

In [ ]:
# importlib: 사용자가 파이썬의 import 시스템과 상호작용하기 위한 API를 제공하는 내장 라이브러리
# 사용자는 import 함수의 구현 내용과, 실행에 필요한 하위 함수들을 이용해 필요에 맞는 임포터를 직접 구현하거나 임포트 관련 기능들을 자유롭게 사용할 수 있음
# importlib.metadata: 설치된 패키지 메타 데이터에 대한 접근을 제공하는 라이브러리.
# 해당 코드 블럭에서는 importlib.metadata 안에 있는 version() 함수를 이용하여 pixellib 라이브러리의 버전을 확인

from importlib.metadata import version
import cv2
import pixellib

print(cv2.__version__)
print(version('pixellib'))

### Step 1. 인물모드 직접 해 보기
---
여러분의 셀카를 이용해서 오늘 배운 내용을 수행해 봅시다. 아래와 같은 이미지를 얻어야 합니다. 최소 3장 이상의 인물모드 사진을 만들어 봅시다.
인물이 주인공이 아닌, 귀여운 고양이에 대한 아웃포커싱 사진도 만들어 볼 수 있을 것입니다. 시맨틱 세그멘테이션 스텝에서 힌트를 찾아봅시다.
배경을 blur하는 인물모드 사진이 아니라 배경사진을 다른 이미지로 교체하는 크로마키 배경 합성을 시도해 볼 수도 있을 것입니다. 여러분만의 환상적인 사진을 만들어 보면 어떨까요?

### Step 2. 사진에서 문제점 찾기
---
그러나 여러분들의 사진에는 무언가 문제점이 있을 것입니다.

아래 사진에도 문제점이 몇 가지 있었습니다. 예를 들어 뒤에 걸린 옷이 인물 영역에 포함되어 blur되지 않고 나온다던가 하는 경우입니다. ㅠㅠ
 그 외 다른 문제들이 눈에 띄시나요? 아래 사진에는 이상한 점이 최소 2개 이상 더 있습니다. 어떤 문제가 있는지 찾아서 아래 사진처럼 표시해 봅시다.

추가로 여러분이 만들어 낸 인물 모드 사진 중 하나에서도 이상한 위치를 찾아 아래 사진처럼 표시해 봅시다. 표시한 이미지들을 jupyter notebook에 포함하여 제출해 주세요.

### Step 3. 해결 방법을 제안해 보기
---
아주 간단히 멋진 카메라 앱 서비스를 만들어 출시할 수 있을 것 같았지만, 지금 같은 완성도로는 어림도 없습니다. 소비자들의 눈높이는 그리 호락호락하지 않고, 엔지니어인 여러분들은 답을 찾아내야 합니다.

생각해 볼 만한 주제를 아래에 몇 가지 제시해 봅니다. 아래를 읽어본 후, 여러분이 만든 인물모드 사진의 문제점을 해결할 방안을 정리하여 제안해 주시기 바랍니다.
단순히 'XXX 기술을 사용한다.' 정도의 선언적 솔루션이 아니라, 여러분들이 선택한 기술이 DeepLab 모델의 Semantic Segmentation 이 만들어 낸 Mask 영역에 어떻게 적용되어 문제점을 보완하게 되는지의 메커니즘이 포함된 솔루션이어야 합니다.

세그멘테이션의 한계

Semantic segmentation의 부정확성이 여러 가지 문제를 발생시키는 주요 원인입니다. 피사계심도를 이용한 보케(아웃포커싱) 효과는 말 그대로 심도를 표현하기 때문에 초점이 잡힌 거리를 광학적으로 아주 섬세하게 구별(segmentation) 하지만 이를 따라 한 semantic segmentation 모듈은 정확도가 1.00 이 되지 않는 한 완벽히 구현하기는 힘듭니다.

피사계 심도 이해하기

우선 피사계심도의 개념부터 명확히 이해해 봅시다.

- 아웃포커싱 하는 법 (https://m.blog.naver.com/typs6301/222172333739)
- 얕은 피사계 심도 촬영의 이해 (https://www.adobe.com/kr/creativecloud/photography/discover/shallow-depth-of-field.html)

3D Camera 활용하기

이미지를 2D 방식으로 받아들이는 한계를 넘어 3D로 인식하려는 시도는 꽤 오래전부터 계속되어 왔습니다. 기술도 이제는 상당한 수준에 이르렀는데요. 크게는 스테레오 비전, ToF 방식 등이 자주 사용됩니다.
 하드웨어 이미지 센서를 통한 3D 이미징 기술이 많이 발전되었는데요. 아래 참고 자료를 통해 최신 기술을 살펴봐도 좋습니다.

- 3D 이미지센서 (https://news.skhynix.co.kr/post/next-gen-3d)

소프트웨어 기술 활용하기

하드웨어 개발과 생산에는 비용이 많이 들기 때문에 제한된 하드웨어를 이용하면서 소프트웨어로 그 한계를 극복하려는 노력도 많습니다. 구글의 struct2Depth가 대표적인 예일 것 같습니다.

- Unsupervised Learning of Depth and Ego-Motion: A Structured Approach (https://sites.google.com/view/struct2depth)

딥러닝이 발전한 지금은 더 다양한 기술이 개발되어 있으니 잘 탐색해 보세요.

다른 기술과 융합해 보기

구글 pixel4에는 IR(적외선) 카메라가 달려있는데요. 물체의 온도를 측정하는 IR 카메라와 3D 이미지는 크게 관련이 없을 것 같지만, 이를 통해 보다 멋진 3d depth sensing이 가능하다는군요.

- uDepth: Real-time 3D Depth Sensing on the Pixel 4 (https://research.google/blog/udepth-real-time-3d-depth-sensing-on-the-pixel-4/)

이렇게 카메라와 무관한 다른 기술과 융합하는 것도 좋은 해결책이 될 수 있습니다. 한 번 고민해 보세요!

이 외에도 다른 방식으로 접근해도 좋습니다. 엔지니어로서 본인이 선택한 솔루션을 잘 설명해 주세요. 가능하다면 순서도(Flow Chart)를 활용하면 좋습니다. 멋진 아이디어를 기대하겠습니다!!

## 4-2. 실습

### 아웃포커싱 인물사진
---

In [ ]:
import os
import urllib
import cv2
import numpy as np
from pixellib.semantic import semantic_segmentation
from matplotlib import pyplot as plt

In [ ]:
img_path1 = os.getenv('HOME')+'/aiffel/workplace/240605-human-segmentation-practice/images/human.png'
img_orig1 = cv2.imread(img_path1) 

print(img_orig1.shape)

plt.imshow(cv2.cvtColor(img_orig1, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
model_dir = os.getenv('HOME')+'/aiffel/workplace/240605-human-segmentation-practice/models' 

model_file = os.path.join(model_dir, 'deeplabv3_xception_tf_dim_ordering_tf_kernels.h5') 

model_url = 'https://github.com/ayoolaolafenwa/PixelLib/releases/download/1.1/deeplabv3_xception_tf_dim_ordering_tf_kernels.h5' 

urllib.request.urlretrieve(model_url, model_file)

In [ ]:
model = semantic_segmentation() 
model.load_pascalvoc_model(model_file)

In [ ]:
segvalues1, output1 = model.segmentAsPascalvoc(img_path1)

In [ ]:
LABEL_NAMES = [
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
]
len(LABEL_NAMES)

In [ ]:
plt.imshow(output1)
plt.show()

In [ ]:
segvalues1

In [ ]:
for class_id in segvalues1['class_ids']:
    print(LABEL_NAMES[class_id])

In [ ]:
colormap = np.zeros((256, 3), dtype = int)
ind = np.arange(256, dtype=int)

for shift in reversed(range(8)):
    for channel in range(3):
        colormap[:, channel] |= ((ind >> channel) & 1) << shift
    ind >>= 3

colormap[:20]

In [ ]:
colormap[15]

In [ ]:
seg_color1 = (128,128,192)

In [ ]:
seg_map1 = np.all(output1==seg_color1, axis=-1) 
print(seg_map1.shape) 
plt.imshow(seg_map1, cmap='gray')
plt.show()

In [ ]:
img_show1 = img_orig1.copy()
img_mask1 = seg_map1.astype(np.uint8) * 255
color_mask1 = cv2.applyColorMap(img_mask1, cv2.COLORMAP_JET)
img_show1 = cv2.addWeighted(img_show1, 0.6, color_mask1, 0.4, 0.0)

plt.imshow(cv2.cvtColor(img_show1, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
img_orig_blur1 = cv2.blur(img_orig1, (50,50))
plt.imshow(cv2.cvtColor(img_orig_blur1, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
img_mask_color1 = cv2.cvtColor(img_mask1, cv2.COLOR_GRAY2BGR)

img_bg_mask1 = cv2.bitwise_not(img_mask_color1)

img_bg_blur1 = cv2.bitwise_and(img_orig_blur1, img_bg_mask1)
plt.imshow(cv2.cvtColor(img_bg_blur1, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
img_concat1 = np.where(img_mask_color1==255, img_orig1, img_bg_blur1)
plt.imshow(cv2.cvtColor(img_concat1, cv2.COLOR_BGR2RGB))
plt.show()

- 다리 한 쪽이 뒤쪽에 있어 색깔이 어둡게 잡혀 background로 인식이 되었다.

-> 후처리기법을 통해 모델이 예측한 마스크를 개선할 수 있다. 특히, Conditional Random Fields (CRF)를 사용하면 객체 경계에서의 잘못된 분할을 수정할 수 있다.

### 아웃포커싱 동물사진
---

In [ ]:
img_path2 = os.getenv('HOME')+'/aiffel/workplace/240605-human-segmentation-practice/images/dog.png'
img_orig2 = cv2.imread(img_path2) 

print(img_orig1.shape)

plt.imshow(cv2.cvtColor(img_orig2, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
segvalues2, output2 = model.segmentAsPascalvoc(img_path2)

In [ ]:
plt.imshow(output2)
plt.show()

In [ ]:
segvalues2

In [ ]:
for class_id in segvalues2['class_ids']:
    print(LABEL_NAMES[class_id])

In [ ]:
colormap = np.zeros((256, 3), dtype = int)
ind = np.arange(256, dtype=int)

for shift in reversed(range(8)):
    for channel in range(3):
        colormap[:, channel] |= ((ind >> channel) & 1) << shift
    ind >>= 3

colormap[:20]

In [ ]:
colormap[12]

In [ ]:
seg_color2 = (128,0,64)

In [ ]:
seg_map2 = np.all(output2==seg_color2, axis=-1) 
print(seg_map2.shape) 
plt.imshow(seg_map2, cmap='gray')
plt.show()

In [ ]:
img_show2 = img_orig2.copy()
img_mask2 = seg_map2.astype(np.uint8) * 255
color_mask2 = cv2.applyColorMap(img_mask2, cv2.COLORMAP_JET)
img_show2 = cv2.addWeighted(img_show2, 0.6, color_mask2, 0.4, 0.0)

plt.imshow(cv2.cvtColor(img_show2, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
img_orig_blur2 = cv2.blur(img_orig2, (50,50))
plt.imshow(cv2.cvtColor(img_orig_blur2, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
img_mask_color2 = cv2.cvtColor(img_mask2, cv2.COLOR_GRAY2BGR)

img_bg_mask2 = cv2.bitwise_not(img_mask_color2)

img_bg_blur2 = cv2.bitwise_and(img_orig_blur2, img_bg_mask2)
plt.imshow(cv2.cvtColor(img_bg_blur2, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
img_concat2 = np.where(img_mask_color2==255, img_orig2, img_bg_blur2)
plt.imshow(cv2.cvtColor(img_concat2, cv2.COLOR_BGR2RGB))
plt.show()

- 피사체와 배경의 색차이가 극명하기에 잘 진행되었다.

### 배경전환 크로마키
---

In [ ]:
img_path3 = os.getenv('HOME')+'/aiffel/workplace/240605-human-segmentation-practice/images/human_chrom.png'
img_path4 = os.getenv('HOME')+'/aiffel/workplace/240605-human-segmentation-practice/images/background_chrom.png'
img_orig3 = cv2.imread(img_path3)
img_orig4 = cv2.imread(img_path4)

print(img_orig3.shape)
print(img_orig4.shape)

In [ ]:
plt.imshow(cv2.cvtColor(img_orig3, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
plt.imshow(cv2.cvtColor(img_orig4, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
segvalues3, output3 = model.segmentAsPascalvoc(img_path3)
segvalues4, output4 = model.segmentAsPascalvoc(img_path4)

In [ ]:
plt.imshow(output3)
plt.show()

In [ ]:
plt.imshow(output4)
plt.show()

In [ ]:
for class_id in segvalues3['class_ids']:
    print(LABEL_NAMES[class_id])

In [ ]:
for class_id in segvalues4['class_ids']:
    print(LABEL_NAMES[class_id])

In [ ]:
seg_color3 = (128,128,192)

In [ ]:
seg_map3 = np.all(output3==seg_color3, axis=-1) 
print(seg_map3.shape) 
plt.imshow(seg_map3, cmap='gray')
plt.show()

In [ ]:
img_show3 = img_orig3.copy()
img_mask3 = seg_map3.astype(np.uint8) * 255
color_mask3 = cv2.applyColorMap(img_mask3, cv2.COLORMAP_JET)
img_show3 = cv2.addWeighted(img_show3, 0.6, color_mask3, 0.4, 0.0)

plt.imshow(cv2.cvtColor(img_show3, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
img_mask_color3 = cv2.cvtColor(img_mask3, cv2.COLOR_GRAY2BGR)

img_bg_mask3 = cv2.bitwise_not(img_mask_color3)

plt.imshow(cv2.cvtColor(img_bg_mask3, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
img_orig4_resized = cv2.resize(img_orig4, (img_orig3.shape[1], img_orig3.shape[0]))

In [ ]:
img_concat3 = np.where(img_mask_color3==255, img_orig3, img_orig4_resized)
plt.imshow(cv2.cvtColor(img_concat3, cv2.COLOR_BGR2RGB))
plt.show()

- 누끼가 잘 따졌다. 만족스럽다.